<a href="https://colab.research.google.com/github/SAQLAIN-MUSHTAQUE/HassanZindani-ResumeScreening/blob/main/Resume_ScreeningV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CV Analyzer

## module install

In [ ]:
!apt-get update #Update the library of Linux
!apt-get install -y poppler-utils #Store temporary data of another library (PDF2image)
# !apt-get install -y libreoffice

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,105 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
G

In [ ]:
! pip install langchain_community docx2txt pinecone-client tiktoken langchain langchain-openai pypdf rapidocr-onnxruntime pdf2image loguru openai tiktoken langchain asyncio aiohttp  -qU

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.1/387.1 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━

## Load Module

In [ ]:
import os #Basic module
import requests
import asyncio #Making a parallel programming
import aiohttp
from concurrent.futures import ThreadPoolExecutor
from pdf2image import convert_from_path #PDF2image conversion
from PIL import Image #PDF2image conversion
import base64 #image format into binary format
from langchain_community.document_loaders import Docx2txtLoader
import io
import tempfile
from google.colab import userdata
from openai import AsyncOpenAI
from typing import Dict, List, Tuple, Any
from loguru import logger
import tiktoken
from google.colab import files
import json
import re
import uuid
import concurrent.futures
import nest_asyncio
import pandas as pd
from pinecone import  Pinecone, ServerlessSpec
from langchain_community.document_loaders.dataframe import DataFrameLoader
from langchain_openai import OpenAIEmbeddings
from itertools import islice
from typing import Iterable, Iterator, List, Any, TypeVar, Dict, Tuple, Union, Optional
from collections import defaultdict

## OpenAI Credential

In [ ]:
# OpenAI Client
client = AsyncOpenAI(api_key = userdata.get('OPENAI_API_KEY'))

## **PDF Reader Using OpenAI Vision** (multiple file)

### PDF to Image

In [ ]:
def pdf_page_to_image(pdf_path):
    """Convert PDF pages to images."""
    try:
      pdf = pdf_path.name
    except:
      pdf = pdf_path

    pages = convert_from_path(pdf, 300)  # 300 DPI for high quality
    return pages

### OpenAI Vision Function

In [ ]:
async def encode_image(image_path):
    """Encode image as a base64 string."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

async def count_tokens_async(text, model="gpt-4o-mini"):
    """Count the number of tokens in a text using tiktoken."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

async def send_image_to_openai_async(image_path, api_key, model="gpt-4o-mini"):
    """Send an image to OpenAI API and receive the extracted text."""
    base64_image = await encode_image(image_path)

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    prompt = """Extract the exact text from the provided image without adding any additional lines at the beginning.
    """

    payload = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 1200
    }

    async with aiohttp.ClientSession() as session:
        async with session.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload) as response:
            response_data = await response.json()
            output_text = response_data['choices'][0]['message']['content']

    # Calculate tokens used
    input_tokens = await count_tokens_async(prompt, model=model)
    output_tokens = await count_tokens_async(output_text, model=model)
    total_tokens = input_tokens + output_tokens

    # Pricing details
    cost_per_input_token = 0.150 / 1_000_000  # $ per input token
    cost_per_output_token = 0.600 / 1_000_000  # $ per output token
    cost_per_image = 0.01275  # $ per image (300x300 px)

    # Calculate total cost
    input_cost = input_tokens * cost_per_input_token
    output_cost = output_tokens * cost_per_output_token
    total_cost = cost_per_image + input_cost + output_cost

    return {
        "content": output_text,
        "total_tokens": total_tokens,
        "total_cost": total_cost
    }

### Process

In [ ]:
async def process_image_text(images, api_key, vision_model="gpt-4o-mini"):
    """Process images and extract text using OpenAI Vision API asynchronously."""
    all_text = ""
    total_tokens = 0
    total_cost = 0

    async def process_single_image(image, image_index):
        with tempfile.NamedTemporaryFile(delete=False, suffix='.png') as temp_file:
            image.save(temp_file, format="PNG")
            temp_file_path = temp_file.name

        result = await send_image_to_openai_async(temp_file_path, api_key, vision_model)
        os.remove(temp_file_path)  # Remove the temporary file after processing

        return image_index, result["content"], float(result["total_tokens"]), float(result["total_cost"])

    tasks = []
    for i, image in enumerate(images, 1):  # Start image index from 1 for ordering
        tasks.append(process_single_image(image, i))

    results = await asyncio.gather(*tasks)

    # Sort the results by image_index
    results.sort(key=lambda x: x[0])

    # Accumulate results
    for image_index, text, tokens, cost in results:
        all_text += text + "\n\n"
        total_tokens += tokens
        total_cost += cost

    return all_text, total_tokens, total_cost

## MS word loader

In [ ]:
def load_docx(file_path):
    loader = Docx2txtLoader(file_path)
    loader =  loader.load()
    text = ""
    for doc in loader:
        text += doc.page_content
    return text


## Pre-processing using LLM

## Chat Completion call

In [ ]:
# System prompt
info_system_prompt = """You are an data expert extractor, provided with raw text extracted from CVs or resumes.
Your task is to search the entire text and extract highly accurate data for the given fields.

The fields are:
1. full_name: Full name of the candidate.
2. contact_number: List the contact number of the candidate, if it is available otherwise None.
3. emial: Email of the candidate, if it is available otherwise None.
4. address: Address of the candidate, if it is available otherwise None.
5. highest_degree: Highest degree of the candidate. If there is no such degree then return None.
6. bachelors_degree: Bachelors degree or undergraduate degree or associate degree with the number of years, if available. If there is no such degree then return None.
7. masters_degree: Masters degree or postgraduate degree with the number of years, if available. If there is no such degree then return None.
8. certifications: All the certifications of the candidate, if available. Return the cetificate name, organization (if available), like: [<certificate1-organization>, <certificate2-organization>, <certificate3>, ...]. If there is no such degree then return None.
9. work_experience: List all the work experience of the candidate in reverse chronological order. Include job titles, company names, the dates of employment and the total duration in that company. The response should be in the list of dictionaries format, where the job title should be the key of most outer dictionaries. If there is no work experience then return None.
10. work_experience_responsibilities: A detailed account of the candidate's previous job roles, listed in reverse chronological order. Include job titles, company names, and bullet points outlining responsibilities and achievements.The response should be in the list of dictionaries format, where the job title should be the key of most outer dictionaries. If there is no work experience then return None.
11. total_years_of_experience: The list of field with the total years of experience of the candidate. If there is no experience then return None. The response of this information should comma separated in the string like: "<field1>:<duration of experience, <field2>:<duration of experience>, ...".
12. skills: A section highlighting both hard skills (technical abilities) and soft skills (interpersonal abilities) that candidate mentioned. If there is no skills then return None.
13. linkedin_links: The linkedin profile link of the candidate, if available. If there is no linkedin link then return None.
14. achievements_and_awards: A list of achievements and awards earned by the candidate. If there is no achievements and awards then return None.
15. projects: A list of the personal or professional projects that the candidate made. If there is no projects then return None.
16. publications: A list of the articles or papers that candidate have authored. If there is no publications then return None .
17. gitHub_link: The github link of the candidate, if available. If there is no github link then return None.
18. Website_link:The portfolio website link of the candidate, if available. If there is no website link then return None.
19. languages: A list of languages that candidate speak, along with your proficiency levels. The value this field should be the list dictionary, where every dictionary has the language as the key and the proficiency level as the value. If there is no languages then return None.
20. extra_info: All the extra information from resume/CV that has not covered in the above fields. If there is no extra information then return None.


**Output format:**
Return your response in JSON format using the structure below:

{{  "full_name": "" || None,
    "contact_number": [] || None,
    "emial": "" || None,
    "address": "" || None,
    "highest_degree": "" || None,
    "bachelors_degree": "" || None,
    "masters_degree": "" || None,
    "certifications": [] || None,
    "work_experience": [{}] || None,
    work_experience_responsibilities: [{}] || None,
    "total_years_of_experience": "" || None,
    "skills": [] || None,
    "linkedin_links": "" || None,
    "achievements_and_awards": [] || None,
    "projects": [] || None,
    "publications": [] || None,
    "gitHub_link": "" || None,
    "Website_link": "" || None,
    "languages": [{}] || None,
    "extra_info": "" || None
}}
"""

In [ ]:
async def extract_information(client, extracted_text, llm_model = "gpt-4o-2024-08-06"):


   # User message
    user_message = f"""Extract the information from the CV/Resume tesxt: {extracted_text}"
    """

    # Generate the completion
    completion = await client.chat.completions.create(
        model=llm_model,
        messages=[
            {"role": "system", "content": info_system_prompt},
            {"role": "user", "content": user_message},
        ],
        response_format={ "type": "json_object" },
    )

    # Get the response message
    message = completion.choices[0].message.content

    return message

## Main orchestration Process

In [ ]:
# Processing Function single file
async def process_single_file(indx: int, filename: str, api_key: str, vision_model: str, llm_model: str) -> Tuple[str, Dict[str, str]]:
    """Asynchronously process a single file by converting to images and extracting text."""
    src_path = os.path.join("/content", filename)
    dest_path = os.path.join("/content/CV_folder", filename)
    os.rename(src_path, dest_path)

    # Get the file path
    file_path = dest_path
    _, ext = os.path.splitext(file_path)
    ext = ext.lower()

    total_tokens = 0.0
    total_cost = 0.0

    if ext == ".docx":
        extracted_text = load_docx(file_path)
    elif ext == ".pdf":
      # Convert PDF pages to images
      images = pdf_page_to_image(file_path)
      print(f"Conversion for file {filename} has been done!")

      # Extract text from images asynchronously using openai vision model
      extracted_text, total_tokens, total_cost = await process_image_text(images, api_key, vision_model=vision_model)
      print (f"Text extraction for file {filename} is: {extracted_text}!")

    # Key (for each file) to store extracted_data
    key = f"file_ {str(indx + 1)}_raw_data"


    # Extracting Pre-Processing data from raw string using OpenAI call function and the model that we are using: 4o-mini
    JSON = await extract_information(client, extracted_text, llm_model= llm_model)

    # Convert to json
    extracted_data = json.loads(JSON)

    total_tokens += 3500
    total_cost += 0.02

    extracted_data['file_name'] = file_path

    result = {
        'filepath': dest_path,
        'extracted_text': extracted_text,
        'extracted_data': extracted_data,
        'total_tokens': total_tokens ,
        'total_cost': total_cost
    }

    return key, result

# Main orcheshtration process (engine first part)
async def main_process(api_key: str, vision_model: str = "gpt-4o-mini", llm_model= "gpt-4o-2024-08-06"):
    """Main function to orchestrate the entire process asynchronously."""

    # Upload the file
    uploaded = files.upload()

    # Create the CV_folder if it doesn't exist
    os.makedirs('/content/CV_folder', exist_ok=True)

    # Initiate variables
    raw_string_extraction = {}
    all_extracted_data = []
    total_tokens = 0.0
    total_cost = 0.0

    # List to hold tasks for processing each file concurrently
    tasks = []

    # Move the uploaded files to /content/CV_folder and process them concurrently
    for indx, filename in enumerate(uploaded.keys()):
        task = process_single_file(indx, filename, api_key, vision_model, llm_model)
        tasks.append(task)

    # Run all the tasks concurrently and gather the results
    results = await asyncio.gather(*tasks)
    # print(results)

    # Process the results
    for key, result in results:
        raw_string_extraction[key] = result['extracted_text']
        all_extracted_data.append(result['extracted_data'])
        total_tokens += result['total_tokens']
        total_cost += result['total_cost']


    # DataFrame of both of the list
    extracted_data_df = pd.DataFrame(all_extracted_data)

    return raw_string_extraction, extracted_data_df, total_tokens, total_cost

In [ ]:
raw_string_extraction, extracted_data_df, total_tokens, total_cost = await main_process(userdata.get('OPENAI_API_KEY'))

TypeError: 'NoneType' object is not subscriptable

In [ ]:
df_data = extracted_data_df
df_data

,full_name,contact_number,emial,address,highest_degree,bachelors_degree,masters_degree,certifications,work_experience,work_experience_responsibilities,...,skills,linkedin_links,achievements_and_awards,projects,publications,gitHub_link,Website_link,languages,extra_info,file_name
0,Affan Sami,[+92 310 284378],affansamiofficial@gmail.com,Karachi,BS-Accounting & Finance,"BS-Accounting & Finance – IoBM, Karachi - Clas...",None,None,[{'Finance Associate': {'company': 'Magnus Cap...,[{'Finance Associate': {'company': 'Magnus Cap...,...,"[Financial Statement Analysis, Peachtree, Fina...",None,None,None,None,None,None,None,"Date and Place of Birth: 09/05/1999, Karachi",/content/CV_folder/Affan Sami resume.pdf
1,H.M. Saqlain Mushtaque,[+92 301 2335058],mushtaqesaqlain@gmail.com,"Karachi, Pakistan",M.Sc. in GIS & Remote Sensing,"B.Sc. (Hons.) in GIS & Remote Sensing, 4 years","M.Sc. in GIS & Remote Sensing, 1 year",[Hands-on Introduction to Linux Commands and S...,[{'Assistant Consultant and Back Office Worker...,[{'Assistant Consultant and Back Office Worker...,...,"[Python, SQL and NoSQL, Linux, GitHub, Data St...",linkedin.com/in/saqlain-mushtaque-065058b9/,None,"[{'Loan Eligibility Prediction': ['Pandas, mat...",None,github.com/SAQLAIN-MUSHTAQUE,None,"[{'Urdu': 'Native or Bilingual Proficiency'}, ...",None,/content/CV_folder/H.M. Saqlain Mushtaque Resu...
2,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,/content/CV_folder/Murtaza_Resume.pdf
3,MUHAMMAD SHARJEEL MUSHTAQUE,[03012442289],sharjeelmushtaque3@gmail.com,Karachi,BS Data Science,"BS Data Science, 4 years",None,"[Programming with R, Google Analytics]","[{'Cro': {'company': 'Tracking world', 'dates'...","[{'Cro': {'company': 'Tracking world', 'respon...",...,"[Algorithms, Big Data, Big data analytics, Dat...",https://www.linkedin.com/in/sharjeel-mushtaque,None,[Train a model using Decision Tree and Naïve B...,None,None,None,"[{'English': 'Advanced'}, {'Urdu': 'Native'}]",Data Scientist Highly motivated and results-dr...,/content/CV_folder/Sharjeel CV.pdf
4,SYED MUHAMMAD GHUFFRAN,[+92347-2360709],ghuffi016@gmail.com,Gulistan e Johar Block 7 Karachi,BE Computer System Engineering,BE Computer System Engineering,None,None,"[{'ML ENGINEER': {'company': 'BRB Group', 'dat...","[{'ML ENGINEER': {'company': 'BRB Group', 'res...",...,"[Embedded Systems: RTOS, Microcontrollers (ARM...",linkedin.com/in/syed-ghuffran-272682169,None,None,None,github.com/SyedGhuffran,None,None,None,/content/CV_folder/SM Ghuffran 2022.pdf


## Checking main orchestration process result

## **Pinecone Upload**



### Pine cone credentials

In [ ]:
PINECONE_API_KEY = userdata.get("PINECONE_API_KEY")
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# OpenAI Embedding model
embedding_model = OpenAIEmbeddings(model = 'text-embedding-3-large')

# Defining Index
pc = Pinecone(PINECONE_API_KEY)


### Delete Pinecone using index

In [ ]:
# Verify deletion by listing indexes again
print([data['name'] for data in pc.list_indexes()])

['cv-analyzer-ba315a78', 'cv-analyzer-6cf73057']


In [ ]:
pc.delete_index([data['name'] for data in pc.list_indexes()][0])

### Creating Pinecone Index

In [ ]:
# Creating Pinecone Index
def generate_unique_index_name(base_name="cv-analyzer"):
    # Generate a unique index name by appending a UUID
    unique_id = uuid.uuid4().hex[:8]  # Get a short unique identifier
    return f"{base_name}_{unique_id}"

# Generate a unique index name (should be save in database)
index_name = generate_unique_index_name().replace('_','-')
print (f"Index name: {index_name}")

# Creating the index with the unique name
pc.create_index(
    name=index_name,
    dimension=3072,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

print(f"Index created with name: {index_name}")

index = pc.Index(index_name)



Index name: cv-analyzer-ba315a78
Index created with name: cv-analyzer-ba315a78


#### If index already created

In [ ]:
index = pc.Index('cv-analyzer-ba315a78')

### Pinecone Upload Function

In [ ]:
T = TypeVar("T")

def batch_iterate(size: int, iterable: Iterable[T]) -> Iterator[List[T]]:
    """Utility batching function."""
    it = iter(iterable)
    while True:
        chunk = list(islice(it, size))
        if not chunk:
            return
        yield chunk
def upload_data(data, index, embedding_model, namespace):
    embedding_chunk_size = 1500
    batch_size = 70 # size of batch (length of chunks in one go.)

    texts = [chunk.page_content for chunk in data]
    metadata_list = [chunk.metadata for chunk in data]
    ids = [str(i) for i in range(len(texts))]

    for metadata, text in zip(metadata_list, texts):
      metadata["text"] = text

    for i in range(0, len(texts), embedding_chunk_size):
      chunk_texts = texts[i : i + embedding_chunk_size]
      chunk_ids = ids[i : i + embedding_chunk_size]
      chunk_metadatas = metadata_list[i : i + embedding_chunk_size]
      embeddings = embedding_model.embed_documents(chunk_texts)

      # uploading asynchronously
      async_res = [
          index.upsert(
              vectors=batch,
              async_req=True,
              namespace=namespace
          )
          for batch in batch_iterate(
              batch_size, zip(chunk_ids, embeddings, chunk_metadatas)
          )
      ]
      [res.get() for res in async_res]

### Data sort out

In [ ]:
# primary_df = df_data.fillna("null")
backup = df_data.copy()
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 21 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   full_name                         4 non-null      object
 1   contact_number                    4 non-null      object
 2   emial                             4 non-null      object
 3   address                           4 non-null      object
 4   highest_degree                    4 non-null      object
 5   bachelors_degree                  4 non-null      object
 6   masters_degree                    1 non-null      object
 7   certifications                    2 non-null      object
 8   work_experience                   4 non-null      object
 9   work_experience_responsibilities  4 non-null      object
 10  total_years_of_experience         4 non-null      object
 11  skills                            4 non-null      object
 12  linkedin_links            

In [ ]:
address = df_data[df_data['address'].notnull()].fillna("null").astype(str)
address

,full_name,contact_number,emial,address,highest_degree,bachelors_degree,masters_degree,certifications,work_experience,work_experience_responsibilities,...,skills,linkedin_links,achievements_and_awards,projects,publications,gitHub_link,Website_link,languages,extra_info,file_name
0,Affan Sami,['+92 310 284378'],affansamiofficial@gmail.com,Karachi,BS-Accounting & Finance,"BS-Accounting & Finance – IoBM, Karachi (4 years)",null,null,[{'Finance Associate': {'company_name': 'Magnu...,[{'Finance Associate': {'company_name': 'Magnu...,...,"['Financial Statement Analysis', 'Peachtree', ...",null,null,null,null,null,null,null,null,/content/CV_folder/Affan Sami resume.pdf
1,H.M. Saqlain Mushtaque,['+92 301 2335058'],mashtaqesaqlain@gmail.com,"Karachi, Pakistan",M.Sc. in GIS & Remote Sensing,"B.Sc. (Hons.) in GIS & Remote Sensing, 4 years","M.Sc. in GIS & Remote Sensing, 1 year",['Hands-on Introduction to Linux Commands and ...,[{'Assistant Consultant and Back Office Worker...,[{'Assistant Consultant and Back Office Worker...,...,"['Python', 'SQL and NoSQL', 'Linux', 'GitHub',...",linkedin.com/in/saqlain-mushtaque-065058b9/,null,"['Loan Eligibility Prediction - Pandas, matplo...",null,github.com/SAQLAIN-MUSHTAQUE,null,"[{'Urdu': 'Native or Bilingual Proficiency'}, ...","Interests include Cloud Computing, Artificial ...",/content/CV_folder/H.M. Saqlain Mushtaque Resu...
3,Muhammad Sharjeel Mushtaque,['03012442289'],sharjeelmushtaque3@gmail.com,Karachi,BS Data Science,"BS Data Science, Institute of Business Managem...",null,"['Programming with R', 'Google Analytics']","[{'Cro': {'company_name': 'Tracking world', 'l...","[{'Cro': {'company_name': 'Tracking world', 'r...",...,"['Algorithms', 'Big Data', 'Big data analytics...",https://www.linkedin.com/in/sharjeel-mushtaque,null,['Machine Learning: Train a model using Decisi...,null,null,null,"[{'English': 'Advanced'}, {'Urdu': 'Native'}]","Passions: Gaming anime, Outing, Snooker",/content/CV_folder/Sharjeel CV.pdf
4,SYED MUHAMMAD GHUFFRAN,['+92347-2360709'],ghuffi016@gmail.com,Gulistan e Johar Block 7 Karachi,BE Computer System Engineering,"BE Computer System Engineering, 2019 - 2023",null,null,"[{'ML ENGINEER': {'company': 'BRB Group', 'dat...","[{'ML ENGINEER': {'company': 'BRB Group', 'res...",...,"['Embedded Systems: RTOS, Microcontrollers (AR...",linkedin.com/in/syed-ghuffran-272682169,null,null,null,github.com/SyedGhuffran,null,null,As a Computer Systems Engineer registered with...,/content/CV_folder/SM Ghuffran 2022.pdf


In [ ]:
highest_degree = df_data[df_data['highest_degree'].notnull()].fillna("null").astype(str)
highest_degree

,full_name,contact_number,emial,address,highest_degree,bachelors_degree,masters_degree,certifications,work_experience,work_experience_responsibilities,...,skills,linkedin_links,achievements_and_awards,projects,publications,gitHub_link,Website_link,languages,extra_info,file_name
0,Affan Sami,['+92 310 284378'],affansamiofficial@gmail.com,Karachi,BS-Accounting & Finance,"BS-Accounting & Finance – IoBM, Karachi (4 years)",null,null,[{'Finance Associate': {'company_name': 'Magnu...,[{'Finance Associate': {'company_name': 'Magnu...,...,"['Financial Statement Analysis', 'Peachtree', ...",null,null,null,null,null,null,null,null,/content/CV_folder/Affan Sami resume.pdf
1,H.M. Saqlain Mushtaque,['+92 301 2335058'],mashtaqesaqlain@gmail.com,"Karachi, Pakistan",M.Sc. in GIS & Remote Sensing,"B.Sc. (Hons.) in GIS & Remote Sensing, 4 years","M.Sc. in GIS & Remote Sensing, 1 year",['Hands-on Introduction to Linux Commands and ...,[{'Assistant Consultant and Back Office Worker...,[{'Assistant Consultant and Back Office Worker...,...,"['Python', 'SQL and NoSQL', 'Linux', 'GitHub',...",linkedin.com/in/saqlain-mushtaque-065058b9/,null,"['Loan Eligibility Prediction - Pandas, matplo...",null,github.com/SAQLAIN-MUSHTAQUE,null,"[{'Urdu': 'Native or Bilingual Proficiency'}, ...","Interests include Cloud Computing, Artificial ...",/content/CV_folder/H.M. Saqlain Mushtaque Resu...
3,Muhammad Sharjeel Mushtaque,['03012442289'],sharjeelmushtaque3@gmail.com,Karachi,BS Data Science,"BS Data Science, Institute of Business Managem...",null,"['Programming with R', 'Google Analytics']","[{'Cro': {'company_name': 'Tracking world', 'l...","[{'Cro': {'company_name': 'Tracking world', 'r...",...,"['Algorithms', 'Big Data', 'Big data analytics...",https://www.linkedin.com/in/sharjeel-mushtaque,null,['Machine Learning: Train a model using Decisi...,null,null,null,"[{'English': 'Advanced'}, {'Urdu': 'Native'}]","Passions: Gaming anime, Outing, Snooker",/content/CV_folder/Sharjeel CV.pdf
4,SYED MUHAMMAD GHUFFRAN,['+92347-2360709'],ghuffi016@gmail.com,Gulistan e Johar Block 7 Karachi,BE Computer System Engineering,"BE Computer System Engineering, 2019 - 2023",null,null,"[{'ML ENGINEER': {'company': 'BRB Group', 'dat...","[{'ML ENGINEER': {'company': 'BRB Group', 'res...",...,"['Embedded Systems: RTOS, Microcontrollers (AR...",linkedin.com/in/syed-ghuffran-272682169,null,null,null,github.com/SyedGhuffran,null,null,As a Computer Systems Engineer registered with...,/content/CV_folder/SM Ghuffran 2022.pdf


In [ ]:
bachelors_degree = df_data[df_data['bachelors_degree'].notnull()].fillna("null").astype(str)
bachelors_degree

,full_name,contact_number,emial,address,highest_degree,bachelors_degree,masters_degree,certifications,work_experience,work_experience_responsibilities,...,skills,linkedin_links,achievements_and_awards,projects,publications,gitHub_link,Website_link,languages,extra_info,file_name
0,Affan Sami,['+92 310 284378'],affansamiofficial@gmail.com,Karachi,BS-Accounting & Finance,"BS-Accounting & Finance – IoBM, Karachi (4 years)",null,null,[{'Finance Associate': {'company_name': 'Magnu...,[{'Finance Associate': {'company_name': 'Magnu...,...,"['Financial Statement Analysis', 'Peachtree', ...",null,null,null,null,null,null,null,null,/content/CV_folder/Affan Sami resume.pdf
1,H.M. Saqlain Mushtaque,['+92 301 2335058'],mashtaqesaqlain@gmail.com,"Karachi, Pakistan",M.Sc. in GIS & Remote Sensing,"B.Sc. (Hons.) in GIS & Remote Sensing, 4 years","M.Sc. in GIS & Remote Sensing, 1 year",['Hands-on Introduction to Linux Commands and ...,[{'Assistant Consultant and Back Office Worker...,[{'Assistant Consultant and Back Office Worker...,...,"['Python', 'SQL and NoSQL', 'Linux', 'GitHub',...",linkedin.com/in/saqlain-mushtaque-065058b9/,null,"['Loan Eligibility Prediction - Pandas, matplo...",null,github.com/SAQLAIN-MUSHTAQUE,null,"[{'Urdu': 'Native or Bilingual Proficiency'}, ...","Interests include Cloud Computing, Artificial ...",/content/CV_folder/H.M. Saqlain Mushtaque Resu...
3,Muhammad Sharjeel Mushtaque,['03012442289'],sharjeelmushtaque3@gmail.com,Karachi,BS Data Science,"BS Data Science, Institute of Business Managem...",null,"['Programming with R', 'Google Analytics']","[{'Cro': {'company_name': 'Tracking world', 'l...","[{'Cro': {'company_name': 'Tracking world', 'r...",...,"['Algorithms', 'Big Data', 'Big data analytics...",https://www.linkedin.com/in/sharjeel-mushtaque,null,['Machine Learning: Train a model using Decisi...,null,null,null,"[{'English': 'Advanced'}, {'Urdu': 'Native'}]","Passions: Gaming anime, Outing, Snooker",/content/CV_folder/Sharjeel CV.pdf
4,SYED MUHAMMAD GHUFFRAN,['+92347-2360709'],ghuffi016@gmail.com,Gulistan e Johar Block 7 Karachi,BE Computer System Engineering,"BE Computer System Engineering, 2019 - 2023",null,null,"[{'ML ENGINEER': {'company': 'BRB Group', 'dat...","[{'ML ENGINEER': {'company': 'BRB Group', 'res...",...,"['Embedded Systems: RTOS, Microcontrollers (AR...",linkedin.com/in/syed-ghuffran-272682169,null,null,null,github.com/SyedGhuffran,null,null,As a Computer Systems Engineer registered with...,/content/CV_folder/SM Ghuffran 2022.pdf


In [ ]:
masters_degree = df_data[df_data['masters_degree'].notnull()].fillna("null").astype(str)
masters_degree

,full_name,contact_number,emial,address,highest_degree,bachelors_degree,masters_degree,certifications,work_experience,work_experience_responsibilities,...,skills,linkedin_links,achievements_and_awards,projects,publications,gitHub_link,Website_link,languages,extra_info,file_name
1,H.M. Saqlain Mushtaque,['+92 301 2335058'],mashtaqesaqlain@gmail.com,"Karachi, Pakistan",M.Sc. in GIS & Remote Sensing,"B.Sc. (Hons.) in GIS & Remote Sensing, 4 years","M.Sc. in GIS & Remote Sensing, 1 year",['Hands-on Introduction to Linux Commands and ...,[{'Assistant Consultant and Back Office Worker...,[{'Assistant Consultant and Back Office Worker...,...,"['Python', 'SQL and NoSQL', 'Linux', 'GitHub',...",linkedin.com/in/saqlain-mushtaque-065058b9/,null,"['Loan Eligibility Prediction - Pandas, matplo...",null,github.com/SAQLAIN-MUSHTAQUE,null,"[{'Urdu': 'Native or Bilingual Proficiency'}, ...","Interests include Cloud Computing, Artificial ...",/content/CV_folder/H.M. Saqlain Mushtaque Resu...


In [ ]:
certifications = df_data[df_data['certifications'].notnull()].fillna("null").astype(str)
certifications

,full_name,contact_number,emial,address,highest_degree,bachelors_degree,masters_degree,certifications,work_experience,work_experience_responsibilities,...,skills,linkedin_links,achievements_and_awards,projects,publications,gitHub_link,Website_link,languages,extra_info,file_name
1,H.M. Saqlain Mushtaque,['+92 301 2335058'],mashtaqesaqlain@gmail.com,"Karachi, Pakistan",M.Sc. in GIS & Remote Sensing,"B.Sc. (Hons.) in GIS & Remote Sensing, 4 years","M.Sc. in GIS & Remote Sensing, 1 year",['Hands-on Introduction to Linux Commands and ...,[{'Assistant Consultant and Back Office Worker...,[{'Assistant Consultant and Back Office Worker...,...,"['Python', 'SQL and NoSQL', 'Linux', 'GitHub',...",linkedin.com/in/saqlain-mushtaque-065058b9/,null,"['Loan Eligibility Prediction - Pandas, matplo...",null,github.com/SAQLAIN-MUSHTAQUE,null,"[{'Urdu': 'Native or Bilingual Proficiency'}, ...","Interests include Cloud Computing, Artificial ...",/content/CV_folder/H.M. Saqlain Mushtaque Resu...
3,Muhammad Sharjeel Mushtaque,['03012442289'],sharjeelmushtaque3@gmail.com,Karachi,BS Data Science,"BS Data Science, Institute of Business Managem...",null,"['Programming with R', 'Google Analytics']","[{'Cro': {'company_name': 'Tracking world', 'l...","[{'Cro': {'company_name': 'Tracking world', 'r...",...,"['Algorithms', 'Big Data', 'Big data analytics...",https://www.linkedin.com/in/sharjeel-mushtaque,null,['Machine Learning: Train a model using Decisi...,null,null,null,"[{'English': 'Advanced'}, {'Urdu': 'Native'}]","Passions: Gaming anime, Outing, Snooker",/content/CV_folder/Sharjeel CV.pdf


In [ ]:
work_experience = df_data[df_data['work_experience'].notnull()].fillna("null").astype(str)
work_experience

,full_name,contact_number,emial,address,highest_degree,bachelors_degree,masters_degree,certifications,work_experience,work_experience_responsibilities,...,skills,linkedin_links,achievements_and_awards,projects,publications,gitHub_link,Website_link,languages,extra_info,file_name
0,Affan Sami,['+92 310 284378'],affansamiofficial@gmail.com,Karachi,BS-Accounting & Finance,"BS-Accounting & Finance – IoBM, Karachi (4 years)",null,null,[{'Finance Associate': {'company_name': 'Magnu...,[{'Finance Associate': {'company_name': 'Magnu...,...,"['Financial Statement Analysis', 'Peachtree', ...",null,null,null,null,null,null,null,null,/content/CV_folder/Affan Sami resume.pdf
1,H.M. Saqlain Mushtaque,['+92 301 2335058'],mashtaqesaqlain@gmail.com,"Karachi, Pakistan",M.Sc. in GIS & Remote Sensing,"B.Sc. (Hons.) in GIS & Remote Sensing, 4 years","M.Sc. in GIS & Remote Sensing, 1 year",['Hands-on Introduction to Linux Commands and ...,[{'Assistant Consultant and Back Office Worker...,[{'Assistant Consultant and Back Office Worker...,...,"['Python', 'SQL and NoSQL', 'Linux', 'GitHub',...",linkedin.com/in/saqlain-mushtaque-065058b9/,null,"['Loan Eligibility Prediction - Pandas, matplo...",null,github.com/SAQLAIN-MUSHTAQUE,null,"[{'Urdu': 'Native or Bilingual Proficiency'}, ...","Interests include Cloud Computing, Artificial ...",/content/CV_folder/H.M. Saqlain Mushtaque Resu...
3,Muhammad Sharjeel Mushtaque,['03012442289'],sharjeelmushtaque3@gmail.com,Karachi,BS Data Science,"BS Data Science, Institute of Business Managem...",null,"['Programming with R', 'Google Analytics']","[{'Cro': {'company_name': 'Tracking world', 'l...","[{'Cro': {'company_name': 'Tracking world', 'r...",...,"['Algorithms', 'Big Data', 'Big data analytics...",https://www.linkedin.com/in/sharjeel-mushtaque,null,['Machine Learning: Train a model using Decisi...,null,null,null,"[{'English': 'Advanced'}, {'Urdu': 'Native'}]","Passions: Gaming anime, Outing, Snooker",/content/CV_folder/Sharjeel CV.pdf
4,SYED MUHAMMAD GHUFFRAN,['+92347-2360709'],ghuffi016@gmail.com,Gulistan e Johar Block 7 Karachi,BE Computer System Engineering,"BE Computer System Engineering, 2019 - 2023",null,null,"[{'ML ENGINEER': {'company': 'BRB Group', 'dat...","[{'ML ENGINEER': {'company': 'BRB Group', 'res...",...,"['Embedded Systems: RTOS, Microcontrollers (AR...",linkedin.com/in/syed-ghuffran-272682169,null,null,null,github.com/SyedGhuffran,null,null,As a Computer Systems Engineer registered with...,/content/CV_folder/SM Ghuffran 2022.pdf


In [ ]:
work_experience_responsibilities = df_data[df_data['work_experience_responsibilities'].notnull()].fillna("null").astype(str)
work_experience_responsibilities

,full_name,contact_number,emial,address,highest_degree,bachelors_degree,masters_degree,certifications,work_experience,work_experience_responsibilities,...,skills,linkedin_links,achievements_and_awards,projects,publications,gitHub_link,Website_link,languages,extra_info,file_name
0,Affan Sami,['+92 310 284378'],affansamiofficial@gmail.com,Karachi,BS-Accounting & Finance,"BS-Accounting & Finance – IoBM, Karachi (4 years)",null,null,[{'Finance Associate': {'company_name': 'Magnu...,[{'Finance Associate': {'company_name': 'Magnu...,...,"['Financial Statement Analysis', 'Peachtree', ...",null,null,null,null,null,null,null,null,/content/CV_folder/Affan Sami resume.pdf
1,H.M. Saqlain Mushtaque,['+92 301 2335058'],mashtaqesaqlain@gmail.com,"Karachi, Pakistan",M.Sc. in GIS & Remote Sensing,"B.Sc. (Hons.) in GIS & Remote Sensing, 4 years","M.Sc. in GIS & Remote Sensing, 1 year",['Hands-on Introduction to Linux Commands and ...,[{'Assistant Consultant and Back Office Worker...,[{'Assistant Consultant and Back Office Worker...,...,"['Python', 'SQL and NoSQL', 'Linux', 'GitHub',...",linkedin.com/in/saqlain-mushtaque-065058b9/,null,"['Loan Eligibility Prediction - Pandas, matplo...",null,github.com/SAQLAIN-MUSHTAQUE,null,"[{'Urdu': 'Native or Bilingual Proficiency'}, ...","Interests include Cloud Computing, Artificial ...",/content/CV_folder/H.M. Saqlain Mushtaque Resu...
3,Muhammad Sharjeel Mushtaque,['03012442289'],sharjeelmushtaque3@gmail.com,Karachi,BS Data Science,"BS Data Science, Institute of Business Managem...",null,"['Programming with R', 'Google Analytics']","[{'Cro': {'company_name': 'Tracking world', 'l...","[{'Cro': {'company_name': 'Tracking world', 'r...",...,"['Algorithms', 'Big Data', 'Big data analytics...",https://www.linkedin.com/in/sharjeel-mushtaque,null,['Machine Learning: Train a model using Decisi...,null,null,null,"[{'English': 'Advanced'}, {'Urdu': 'Native'}]","Passions: Gaming anime, Outing, Snooker",/content/CV_folder/Sharjeel CV.pdf
4,SYED MUHAMMAD GHUFFRAN,['+92347-2360709'],ghuffi016@gmail.com,Gulistan e Johar Block 7 Karachi,BE Computer System Engineering,"BE Computer System Engineering, 2019 - 2023",null,null,"[{'ML ENGINEER': {'company': 'BRB Group', 'dat...","[{'ML ENGINEER': {'company': 'BRB Group', 'res...",...,"['Embedded Systems: RTOS, Microcontrollers (AR...",linkedin.com/in/syed-ghuffran-272682169,null,null,null,github.com/SyedGhuffran,null,null,As a Computer Systems Engineer registered with...,/content/CV_folder/SM Ghuffran 2022.pdf


In [ ]:
total_years_of_experience = df_data[df_data['total_years_of_experience'].notnull()].fillna("null").astype(str)
total_years_of_experience

,full_name,contact_number,emial,address,highest_degree,bachelors_degree,masters_degree,certifications,work_experience,work_experience_responsibilities,...,skills,linkedin_links,achievements_and_awards,projects,publications,gitHub_link,Website_link,languages,extra_info,file_name
0,Affan Sami,['+92 310 284378'],affansamiofficial@gmail.com,Karachi,BS-Accounting & Finance,"BS-Accounting & Finance – IoBM, Karachi (4 years)",null,null,[{'Finance Associate': {'company_name': 'Magnu...,[{'Finance Associate': {'company_name': 'Magnu...,...,"['Financial Statement Analysis', 'Peachtree', ...",null,null,null,null,null,null,null,null,/content/CV_folder/Affan Sami resume.pdf
1,H.M. Saqlain Mushtaque,['+92 301 2335058'],mashtaqesaqlain@gmail.com,"Karachi, Pakistan",M.Sc. in GIS & Remote Sensing,"B.Sc. (Hons.) in GIS & Remote Sensing, 4 years","M.Sc. in GIS & Remote Sensing, 1 year",['Hands-on Introduction to Linux Commands and ...,[{'Assistant Consultant and Back Office Worker...,[{'Assistant Consultant and Back Office Worker...,...,"['Python', 'SQL and NoSQL', 'Linux', 'GitHub',...",linkedin.com/in/saqlain-mushtaque-065058b9/,null,"['Loan Eligibility Prediction - Pandas, matplo...",null,github.com/SAQLAIN-MUSHTAQUE,null,"[{'Urdu': 'Native or Bilingual Proficiency'}, ...","Interests include Cloud Computing, Artificial ...",/content/CV_folder/H.M. Saqlain Mushtaque Resu...
3,Muhammad Sharjeel Mushtaque,['03012442289'],sharjeelmushtaque3@gmail.com,Karachi,BS Data Science,"BS Data Science, Institute of Business Managem...",null,"['Programming with R', 'Google Analytics']","[{'Cro': {'company_name': 'Tracking world', 'l...","[{'Cro': {'company_name': 'Tracking world', 'r...",...,"['Algorithms', 'Big Data', 'Big data analytics...",https://www.linkedin.com/in/sharjeel-mushtaque,null,['Machine Learning: Train a model using Decisi...,null,null,null,"[{'English': 'Advanced'}, {'Urdu': 'Native'}]","Passions: Gaming anime, Outing, Snooker",/content/CV_folder/Sharjeel CV.pdf
4,SYED MUHAMMAD GHUFFRAN,['+92347-2360709'],ghuffi016@gmail.com,Gulistan e Johar Block 7 Karachi,BE Computer System Engineering,"BE Computer System Engineering, 2019 - 2023",null,null,"[{'ML ENGINEER': {'company': 'BRB Group', 'dat...","[{'ML ENGINEER': {'company': 'BRB Group', 'res...",...,"['Embedded Systems: RTOS, Microcontrollers (AR...",linkedin.com/in/syed-ghuffran-272682169,null,null,null,github.com/SyedGhuffran,null,null,As a Computer Systems Engineer registered with...,/content/CV_folder/SM Ghuffran 2022.pdf


In [ ]:
skills = df_data[df_data['skills'].notnull()].fillna("null").astype(str)
skills

,full_name,contact_number,emial,address,highest_degree,bachelors_degree,masters_degree,certifications,work_experience,work_experience_responsibilities,...,skills,linkedin_links,achievements_and_awards,projects,publications,gitHub_link,Website_link,languages,extra_info,file_name
0,Affan Sami,['+92 310 284378'],affansamiofficial@gmail.com,Karachi,BS-Accounting & Finance,"BS-Accounting & Finance – IoBM, Karachi (4 years)",null,null,[{'Finance Associate': {'company_name': 'Magnu...,[{'Finance Associate': {'company_name': 'Magnu...,...,"['Financial Statement Analysis', 'Peachtree', ...",null,null,null,null,null,null,null,null,/content/CV_folder/Affan Sami resume.pdf
1,H.M. Saqlain Mushtaque,['+92 301 2335058'],mashtaqesaqlain@gmail.com,"Karachi, Pakistan",M.Sc. in GIS & Remote Sensing,"B.Sc. (Hons.) in GIS & Remote Sensing, 4 years","M.Sc. in GIS & Remote Sensing, 1 year",['Hands-on Introduction to Linux Commands and ...,[{'Assistant Consultant and Back Office Worker...,[{'Assistant Consultant and Back Office Worker...,...,"['Python', 'SQL and NoSQL', 'Linux', 'GitHub',...",linkedin.com/in/saqlain-mushtaque-065058b9/,null,"['Loan Eligibility Prediction - Pandas, matplo...",null,github.com/SAQLAIN-MUSHTAQUE,null,"[{'Urdu': 'Native or Bilingual Proficiency'}, ...","Interests include Cloud Computing, Artificial ...",/content/CV_folder/H.M. Saqlain Mushtaque Resu...
3,Muhammad Sharjeel Mushtaque,['03012442289'],sharjeelmushtaque3@gmail.com,Karachi,BS Data Science,"BS Data Science, Institute of Business Managem...",null,"['Programming with R', 'Google Analytics']","[{'Cro': {'company_name': 'Tracking world', 'l...","[{'Cro': {'company_name': 'Tracking world', 'r...",...,"['Algorithms', 'Big Data', 'Big data analytics...",https://www.linkedin.com/in/sharjeel-mushtaque,null,['Machine Learning: Train a model using Decisi...,null,null,null,"[{'English': 'Advanced'}, {'Urdu': 'Native'}]","Passions: Gaming anime, Outing, Snooker",/content/CV_folder/Sharjeel CV.pdf
4,SYED MUHAMMAD GHUFFRAN,['+92347-2360709'],ghuffi016@gmail.com,Gulistan e Johar Block 7 Karachi,BE Computer System Engineering,"BE Computer System Engineering, 2019 - 2023",null,null,"[{'ML ENGINEER': {'company': 'BRB Group', 'dat...","[{'ML ENGINEER': {'company': 'BRB Group', 'res...",...,"['Embedded Systems: RTOS, Microcontrollers (AR...",linkedin.com/in/syed-ghuffran-272682169,null,null,null,github.com/SyedGhuffran,null,null,As a Computer Systems Engineer registered with...,/content/CV_folder/SM Ghuffran 2022.pdf


In [ ]:
achievements_and_awards = df_data[df_data['achievements_and_awards'].notnull()].fillna("null").astype(str)
achievements_and_awards

,full_name,contact_number,emial,address,highest_degree,bachelors_degree,masters_degree,certifications,work_experience,work_experience_responsibilities,...,skills,linkedin_links,achievements_and_awards,projects,publications,gitHub_link,Website_link,languages,extra_info,file_name


In [ ]:
projects = df_data[df_data['projects'].notnull()].fillna("null").astype(str)
projects

,full_name,contact_number,emial,address,highest_degree,bachelors_degree,masters_degree,certifications,work_experience,work_experience_responsibilities,...,skills,linkedin_links,achievements_and_awards,projects,publications,gitHub_link,Website_link,languages,extra_info,file_name
1,H.M. Saqlain Mushtaque,['+92 301 2335058'],mashtaqesaqlain@gmail.com,"Karachi, Pakistan",M.Sc. in GIS & Remote Sensing,"B.Sc. (Hons.) in GIS & Remote Sensing, 4 years","M.Sc. in GIS & Remote Sensing, 1 year",['Hands-on Introduction to Linux Commands and ...,[{'Assistant Consultant and Back Office Worker...,[{'Assistant Consultant and Back Office Worker...,...,"['Python', 'SQL and NoSQL', 'Linux', 'GitHub',...",linkedin.com/in/saqlain-mushtaque-065058b9/,null,"['Loan Eligibility Prediction - Pandas, matplo...",null,github.com/SAQLAIN-MUSHTAQUE,null,"[{'Urdu': 'Native or Bilingual Proficiency'}, ...","Interests include Cloud Computing, Artificial ...",/content/CV_folder/H.M. Saqlain Mushtaque Resu...
3,Muhammad Sharjeel Mushtaque,['03012442289'],sharjeelmushtaque3@gmail.com,Karachi,BS Data Science,"BS Data Science, Institute of Business Managem...",null,"['Programming with R', 'Google Analytics']","[{'Cro': {'company_name': 'Tracking world', 'l...","[{'Cro': {'company_name': 'Tracking world', 'r...",...,"['Algorithms', 'Big Data', 'Big data analytics...",https://www.linkedin.com/in/sharjeel-mushtaque,null,['Machine Learning: Train a model using Decisi...,null,null,null,"[{'English': 'Advanced'}, {'Urdu': 'Native'}]","Passions: Gaming anime, Outing, Snooker",/content/CV_folder/Sharjeel CV.pdf


In [ ]:
publications = df_data[df_data['publications'].notnull()].fillna("null").astype(str)
publications

,full_name,contact_number,emial,address,highest_degree,bachelors_degree,masters_degree,certifications,work_experience,work_experience_responsibilities,...,skills,linkedin_links,achievements_and_awards,projects,publications,gitHub_link,Website_link,languages,extra_info,file_name


In [ ]:
Website_link = df_data[df_data['Website_link'].notnull()].fillna("null").astype(str)
Website_link

,full_name,contact_number,emial,address,highest_degree,bachelors_degree,masters_degree,certifications,work_experience,work_experience_responsibilities,...,skills,linkedin_links,achievements_and_awards,projects,publications,gitHub_link,Website_link,languages,extra_info,file_name


In [ ]:
languages = df_data[df_data['languages'].notnull()].fillna("null").astype(str)
languages

,full_name,contact_number,emial,address,highest_degree,bachelors_degree,masters_degree,certifications,work_experience,work_experience_responsibilities,...,skills,linkedin_links,achievements_and_awards,projects,publications,gitHub_link,Website_link,languages,extra_info,file_name
1,H.M. Saqlain Mushtaque,['+92 301 2335058'],mashtaqesaqlain@gmail.com,"Karachi, Pakistan",M.Sc. in GIS & Remote Sensing,"B.Sc. (Hons.) in GIS & Remote Sensing, 4 years","M.Sc. in GIS & Remote Sensing, 1 year",['Hands-on Introduction to Linux Commands and ...,[{'Assistant Consultant and Back Office Worker...,[{'Assistant Consultant and Back Office Worker...,...,"['Python', 'SQL and NoSQL', 'Linux', 'GitHub',...",linkedin.com/in/saqlain-mushtaque-065058b9/,null,"['Loan Eligibility Prediction - Pandas, matplo...",null,github.com/SAQLAIN-MUSHTAQUE,null,"[{'Urdu': 'Native or Bilingual Proficiency'}, ...","Interests include Cloud Computing, Artificial ...",/content/CV_folder/H.M. Saqlain Mushtaque Resu...
3,Muhammad Sharjeel Mushtaque,['03012442289'],sharjeelmushtaque3@gmail.com,Karachi,BS Data Science,"BS Data Science, Institute of Business Managem...",null,"['Programming with R', 'Google Analytics']","[{'Cro': {'company_name': 'Tracking world', 'l...","[{'Cro': {'company_name': 'Tracking world', 'r...",...,"['Algorithms', 'Big Data', 'Big data analytics...",https://www.linkedin.com/in/sharjeel-mushtaque,null,['Machine Learning: Train a model using Decisi...,null,null,null,"[{'English': 'Advanced'}, {'Urdu': 'Native'}]","Passions: Gaming anime, Outing, Snooker",/content/CV_folder/Sharjeel CV.pdf


In [ ]:
extra_info = df_data[df_data['extra_info'].notnull()].fillna("null").astype(str)
extra_info

,full_name,contact_number,emial,address,highest_degree,bachelors_degree,masters_degree,certifications,work_experience,work_experience_responsibilities,...,skills,linkedin_links,achievements_and_awards,projects,publications,gitHub_link,Website_link,languages,extra_info,file_name
1,H.M. Saqlain Mushtaque,['+92 301 2335058'],mashtaqesaqlain@gmail.com,"Karachi, Pakistan",M.Sc. in GIS & Remote Sensing,"B.Sc. (Hons.) in GIS & Remote Sensing, 4 years","M.Sc. in GIS & Remote Sensing, 1 year",['Hands-on Introduction to Linux Commands and ...,[{'Assistant Consultant and Back Office Worker...,[{'Assistant Consultant and Back Office Worker...,...,"['Python', 'SQL and NoSQL', 'Linux', 'GitHub',...",linkedin.com/in/saqlain-mushtaque-065058b9/,null,"['Loan Eligibility Prediction - Pandas, matplo...",null,github.com/SAQLAIN-MUSHTAQUE,null,"[{'Urdu': 'Native or Bilingual Proficiency'}, ...","Interests include Cloud Computing, Artificial ...",/content/CV_folder/H.M. Saqlain Mushtaque Resu...
3,Muhammad Sharjeel Mushtaque,['03012442289'],sharjeelmushtaque3@gmail.com,Karachi,BS Data Science,"BS Data Science, Institute of Business Managem...",null,"['Programming with R', 'Google Analytics']","[{'Cro': {'company_name': 'Tracking world', 'l...","[{'Cro': {'company_name': 'Tracking world', 'r...",...,"['Algorithms', 'Big Data', 'Big data analytics...",https://www.linkedin.com/in/sharjeel-mushtaque,null,['Machine Learning: Train a model using Decisi...,null,null,null,"[{'English': 'Advanced'}, {'Urdu': 'Native'}]","Passions: Gaming anime, Outing, Snooker",/content/CV_folder/Sharjeel CV.pdf
4,SYED MUHAMMAD GHUFFRAN,['+92347-2360709'],ghuffi016@gmail.com,Gulistan e Johar Block 7 Karachi,BE Computer System Engineering,"BE Computer System Engineering, 2019 - 2023",null,null,"[{'ML ENGINEER': {'company': 'BRB Group', 'dat...","[{'ML ENGINEER': {'company': 'BRB Group', 'res...",...,"['Embedded Systems: RTOS, Microcontrollers (AR...",linkedin.com/in/syed-ghuffran-272682169,null,null,null,github.com/SyedGhuffran,null,null,As a Computer Systems Engineer registered with...,/content/CV_folder/SM Ghuffran 2022.pdf


### Pinecone Uploading

In [ ]:
# For Address
address_pc = DataFrameLoader(address, page_content_column="address").load()
upload_data(address_pc, index, embedding_model, "location")

In [ ]:
highest_degree_pc= DataFrameLoader(highest_degree, page_content_column="highest_degree").load()
upload_data(highest_degree_pc, index, embedding_model, "highest_degree")

In [ ]:
bachelors_degree_pc= DataFrameLoader(bachelors_degree, page_content_column="bachelors_degree").load()
upload_data(bachelors_degree_pc, index, embedding_model, "bachelors_degree")

In [ ]:
masters_degree_pc= DataFrameLoader(masters_degree, page_content_column="masters_degree").load()
upload_data(masters_degree_pc, index, embedding_model, "masters_degree")

In [ ]:
certifications_pc= DataFrameLoader(certifications, page_content_column="certifications").load()
upload_data(certifications_pc, index, embedding_model, "certifications")

In [ ]:
work_experience_pc= DataFrameLoader(work_experience, page_content_column="work_experience").load()
upload_data(work_experience_pc, index, embedding_model, "work_experience")

In [ ]:
work_experience_responsibilities_pc= DataFrameLoader(work_experience_responsibilities, page_content_column="work_experience_responsibilities").load()
upload_data(work_experience_responsibilities_pc, index, embedding_model, "work_experience_responsibilities")

In [ ]:
total_years_of_experience_pc= DataFrameLoader(total_years_of_experience, page_content_column="total_years_of_experience").load()
upload_data(total_years_of_experience_pc, index, embedding_model, "total_years_of_experience")

In [ ]:
skills_pc= DataFrameLoader(skills, page_content_column="skills").load()
upload_data(skills_pc, index, embedding_model, "skills")

In [ ]:
achievements_and_awards_pc= DataFrameLoader(achievements_and_awards, page_content_column="achievements_and_awards").load()
upload_data(achievements_and_awards_pc, index, embedding_model, "achievements_and_awards")

In [ ]:
projects_pc= DataFrameLoader(projects, page_content_column="projects").load()
upload_data(projects_pc, index, embedding_model, "projects")

In [ ]:
publications_pc= DataFrameLoader(publications, page_content_column="publications").load()
upload_data(publications_pc, index, embedding_model, "publications")

In [ ]:
languages_pc= DataFrameLoader(languages, page_content_column="languages").load()
upload_data(languages_pc, index, embedding_model, "languages")

In [ ]:
extra_info_pc= DataFrameLoader(extra_info, page_content_column="extra_info").load()
upload_data(extra_info_pc, index, embedding_model, "extra_info")

#### Checking length of any namespace

In [ ]:
stats = index.describe_index_stats()

# Check the namespaces and their vector counts
namespace_stats = stats['namespaces']
print(namespace_stats)

namespace = 'extra_info'
count = namespace_stats.get(namespace,{}).get('vector_count', 0)
print(f"Number of entries in the namespace '{namespace}': {count}")

{'languages': {'vector_count': 2}, 'projects': {'vector_count': 2}, 'certifications': {'vector_count': 2}, 'skills': {'vector_count': 4}, 'masters_degree': {'vector_count': 1}, 'location': {'vector_count': 4}, 'work_experience_responsibilities': {'vector_count': 4}, 'total_years_of_experience': {'vector_count': 4}, 'extra_info': {'vector_count': 3}, 'work_experience': {'vector_count': 4}, 'highest_degree': {'vector_count': 4}, 'bachelors_degree': {'vector_count': 4}}
Number of entries in the namespace 'extra_info': 3


## Query:

In [ ]:
query = """
As a DevOps Engineers at Dubizzle ensures, you will ensure our services are healthy, monitored, automated, and designed to scale. You'll use your background as an operations generalist to work closely with our development teams from the early stages of design all the way through identifying and resolving production issues. You will support a wide range of products focusing on automation, availability and performance, and above all reliability.


Requirements

Serve as a primary point responsible for the overall health, performance, efficiency and capacity of our production environment
Be part of the implementation and design of the systems used to operate Dubizzle, with a focus on automation and maintainability at large scale
Experience in building large CI/CD processes (experience with Github actions & Jenkins is preferred)
Collaboration with the development team on operations-related issues, providing support and acting as stakeholder
Provide fast and reliable development experience for product teams by maintaining production-like environments used for development and testing
Develop tools to effectively monitor custom applications in a large-scale environment
Troubleshoot issues across the entire stack - hardware, software, application and network
Migrate applications off of legacy environments with minimal downtime
Take part in a shared 24x7 on-call rotation
Document system design and procedures
Experience & Qualification

Bachelor's Degree in Computer Science, Engineering or equivalent field and significant experience in DevOps
AWS Solutions Architect certification
4+ years experience in supporting hosted services in a high-volume customer-facing environment
Experience with DevOps tools, processes, culture, distributed systems, supporting mission-critical systems
Experience working on Continuous Integration / Delivery pipelines using tools such as Jenkins and Github actions
Demonstrated ability to write programs / utilities using a high-level programming language like: Go, Ruby, Python as well as Bash
3+ years proven experience in running and managing infrastructure at scale using container orchestration tool kubernetes
Experience in performing migration from traditional technology stack(s) to AWS EKS or similar


Benefits



 fast paced, high performing team
ulticultural environment with over 40 different nationalities
omprehensive Health insurance (Spouse & Children)
ewards & Recognitions
earning & Development opportunities
ym Allowance
aptop Ownership
aid Leaves

Benefits found in job post

Medical insurance
"""

### quary system Prompt

In [ ]:
query_prompt = """
You are an excellent Job Post Analyzer. You will provided a job post and your job is to analyze it and extract the following information.

Information_fields:
job_title: The title of the job. It must be present in the job posting. It should be string "". This information is compulsory and not empty, and if there is no job title, then this is not a valid job post, and in that case, simply return "Invalid Job Post" without searching for the further field's information.
job_description: The description of the job. It must be present in the job posting. It should be string "". If there is no job description, then return None.
required_experience: The experience or the level of the candidate required for the job. This information may describe the level of experience or the duration of experience that candidate must have. If there is more than one required experience, then write the job role with the required number of durations of experience. This information should be the string. This information must be present in the job posting, if it is not present write your remarks from yor understanding.
job_reponsibilities_required: The list of responsibilities or the roles for the given job that candidate must have. This information should be written in comma separated value inside the string. This information must be present in the job posting, if it is not present write your remarks from yor understanding.
job_reponsibilities_optional: The list of responsibilities or the roles for the given job that candidate can have. This information should be written in comma separated value inside the string "". This information may be present in the job posting. If there is no optional responsibilities, then return None.
required_education: The list of education required for the job. This information should be written in comma separated value inside the string. This information must be present in the job posting. Otherwise, write your remarks.
required_degree: The degree or list of qualifications required for the job. This information should be written in comma separated value inside the string "". If there is no degree required, then return None.
required_skills: The list of skills required for the job. This information should be written in comma separated value inside the string "". This information must be present in the job posting, if it is not present write your remarks from yor understanding.
required_location: The requirement of location for the job. This information should be written in comma separated value inside the string "". This information is not compulsory. If there is no location required, then return None.
extra_info: All the other information that are not listed in the above fields. This information should be in detail without leaving any single information that are not addressed above. This information should be written in comma separated value inside the string "". This information may be present in the job posting. If there is no extra information, then return None.

Your output should be in the JSON format:
{{
  "job_title": "",
  "job_description": "" || None,
  "required_experience": "",
  "job_reponsibilities_required": "",
  "job_reponsibilities_optional": "" || None,
  "required_education": "",
  "required_degree": "" || None,
  "required_skills": "" ,
  "required_location": "" || None,
  "extra_info": "" || None
}}

"""

### Pass to the LLM (OpenAI)

In [ ]:
async def extract_query_info(client, query_system_prompt, query, llm_model = "gpt-4o-2024-08-06"):


   # User message
    user_message = f"""Extract the information from the job post text: {query}"""

    # Generate the completion
    completion = await client.chat.completions.create(
        model=llm_model,
        messages=[
            {"role": "system", "content": query_system_prompt},
            {"role": "user", "content": user_message},
        ],
        response_format={ "type": "json_object" },
    )

    # Get the response message
    json_message = completion.choices[0].message.content
    message = json.loads(json_message)

    return message

In [ ]:
extracted_query_info= await  extract_query_info(client, query_system_prompt = query_prompt, query= query, llm_model = "gpt-4o-2024-08-06")

In [ ]:
extracted_query_info

{'job_title': 'DevOps Engineer',
 'job_description': "As a DevOps Engineers at Dubizzle ensures, you will ensure our services are healthy, monitored, automated, and designed to scale. You'll use your background as an operations generalist to work closely with our development teams from the early stages of design all the way through identifying and resolving production issues. You will support a wide range of products focusing on automation, availability and performance, and above all reliability.",
 'required_experience': '4+ years experience in supporting hosted services in a high-volume customer-facing environment, 3+ years proven experience in running and managing infrastructure at scale using container orchestration tool kubernetes',
 'job_reponsibilities_required': 'Serve as a primary point responsible for the overall health, performance, efficiency and capacity of our production environment, Be part of the implementation and design of the systems used to operate Dubizzle, with a 

### Filling the query info in the variables

In [ ]:
job_data = {
    'job_title': extracted_query_info.get('job_title'),
    'job_description': extracted_query_info.get('job_description', ''),
    'required_experience': extracted_query_info.get('required_experience'),
    'job_responsibilities_required': extracted_query_info.get('job_reponsibilities_required'),
    'job_responsibilities_optional': extracted_query_info.get('job_reponsibilities_optional', ''),
    'required_education': extracted_query_info.get('required_education'),
    'required_degree': extracted_query_info.get('required_degree', ''),
    'required_skills': extracted_query_info.get('required_skills'),
    'required_location': extracted_query_info.get('required_location', ''),
    'extra_info': extracted_query_info.get('extra_info', '')
}


## Semantic search

### Query Function

In [ ]:
# Pinecone Query
async def query_pinecone(query: str, namespace: str, index:str, top_k: int , filter: dict = None) -> list:
    """
    Query the Pinecone index with the given parameters.

    Args:
        query: The query string to be embedded and searched.
        namespace: The namespace to query within.
        top_k: The number of top results to return.
        filter: Optional dictionary for metadata filtering.

    Returns:
        list:The query responses from Pinecone.
    """
    # Embed the query string
    loop = asyncio.get_running_loop()
    with ThreadPoolExecutor() as pool:
        # Embedding in thread pool
        embedded = await loop.run_in_executor(pool, embedding_model.embed_query, query)

        # Pinecone query in thread pool
        query_params = {
            "namespace": namespace,
            "vector": embedded,
            "top_k": top_k,
            "include_metadata": True,
        }
        if filter:
            query_params["filter"] = filter

        try:
            responses = await loop.run_in_executor(pool, lambda: index.query(**query_params))
        except Exception as e:
            print(f"Error during Pinecone query: {e}")
            return []

    # Extract essential data
    results = []
    for match in responses["matches"]:
        # print (f"{query} \n {match['id']}")
        results.append({
            "id": match["id"],
            "score": match["score"],
            "metadata": match["metadata"],
            "values":match["values"]
        })
    return results  # Return a list of dictionaries instead of Pinecone matches


In [ ]:
# Semaphore Function
# Limit the number of concurrent requests
semaphore = asyncio.Semaphore(20)  # Adjust the limit based on your requirements

async def query_with_semaphore(*args, **kwargs):
    async with semaphore:
        return await query_pinecone(*args, **kwargs)

### Base Search

In [ ]:
async def filtering_skill_edu_exp(job_data,index):
  # Helper function to handle querying and extracting matching
  async def query_matching(query: str, index:str, namespace: str, top_k: int):
      """
      Queries the Pinecone service with the provided query string, namespace, and top_k value.

      Args:
          query (str): The query string to search for.
          namespace (str): The namespace in Pinecone to query against.
          top_k (int): The number of top results to return.

      Returns:
          list: A list of fund dictionaries returned from Pinecone.
      """
      return await query_pinecone(query=query, index=index, namespace=namespace, top_k=top_k)

  # Calculating top-k using index
  stats = index.describe_index_stats()
  # Check the namespaces and their vector counts
  namespace_stats = stats['namespaces']
  namespace= 'work_experience'
  work_top_k= namespace_stats.get(namespace,{}).get('vector_count', 0)

  # Calculating top-k using index
  stats = index.describe_index_stats()
  # check the namespaces and their vector counts
  namespace_stats = stats['namespaces']
  namespace= 'highest_degree'
  experience_top_k= namespace_stats.get(namespace,{}).get('vector_count', 0)

  # calculating top-k using index
  stats = index.describe_index_stats()
  # check the namespaces and their vector counts
  namespace_stats = stats['namespaces']
  namespace= 'skills'
  skill_top_k= namespace_stats.get(namespace,{}).get('vector_count', 0)

  # Create asynchronous tasks for fund queries
  if 'required_experience' and job_data['required_experience'] != "":
    query1 = job_data['required_experience']
    task1 = []
    task1 += [query_with_semaphore(query = query1, index= index, namespace= namespace, top_k= work_top_k)]
    task1 += [query_with_semaphore(query = query1, index= index, namespace= namespace, top_k= experience_top_k)]
    task1 += [query_with_semaphore(query = query1, index= index, namespace= namespace, top_k= skill_top_k)]

      # 'skill_matching': query_matching(diversity_focused, "funds-diversity-focus", top_k_diversity_focus),
  if 'required_education' and job_data['required_education'] != "":
    query2 = job_data['required_education']
    task2 = []
    task2 += [query_with_semaphore(query = query2, index= index, namespace= namespace, top_k= work_top_k)]
    task2 += [query_with_semaphore(query = query2, index= index, namespace= namespace, top_k= experience_top_k)]
    task2 += [query_with_semaphore(query = query2, index= index, namespace= namespace, top_k= skill_top_k)]

  if 'required_skills' and job_data['required_skills'] != "":
    query3 = job_data['required_skills']
    task3 = []
    task3 += [query_with_semaphore(query = query3, index= index, namespace= namespace, top_k= work_top_k)]
    task3 += [query_with_semaphore(query = query3, index= index, namespace= namespace, top_k= experience_top_k)]
    task3 += [query_with_semaphore(query = query3, index= index, namespace= namespace, top_k= skill_top_k)]

  # Run task1 and task2 concurrently
  results1, results2, results3 = await asyncio.gather(
      asyncio.gather(*task1),  # Gather all results from task1
      asyncio.gather(*task2),  # Gather all results from task2
      asyncio.gather(*task3)   # Gather all results from task3
  )

  final_results = {}

  for r1 in results1[0]:
      file_name = r1['metadata']['file_name']
      if r1['score'] >=0.3:
          if file_name not in final_results:
              final_results[file_name] = {}
              final_results[file_name]['candidate_name'] = r1['metadata']['full_name']
              final_results[file_name]['metadata'] = r1['metadata']
              final_results[file_name]['score'] = [r1['score']]  # Start fresh list for each file
              final_results[file_name]['values'] = [r1['values']]  # Start fresh list for each file
              final_results[file_name]['metadata']['work_experience'] = final_results[file_name]['metadata']['text']
              final_results[file_name]['reason'] = f"The experience matched with the experience of {r1['metadata']['full_name']}"
          else:
              final_results[file_name]['score'].append(r1['score'])
              final_results[file_name]['values'].append(r1['values'])
              final_results[file_name]['reason'] += "\n" + f"The experience matched with the experience of {r1['metadata']['full_name']}"

  for r2 in results2[0]:
      file_name = r2['metadata']['file_name']
      if r2['score'] >=0.3:
          if file_name not in final_results:
              final_results[file_name] = {}
              final_results[file_name]['candidate_name'] = r2['metadata']['full_name']
              final_results[file_name]['metadata'] = r2['metadata']
              final_results[file_name]['score'] = [r2['score']]
              final_results[file_name]['values'] = [r2['values']]
              final_results[file_name]['metadata']['required_education'] = final_results[file_name]['metadata']['text']
              final_results[file_name]['reason'] = f"The education matched with the education of {r2['metadata']['full_name']}"
          else:
              final_results[file_name]['score'].append(r2['score'])
              final_results[file_name]['values'].append(r2['values'])
              final_results[file_name]['reason'] += "\n" + f"The education matched with the education of {r2['metadata']['full_name']}"

  for r3 in results3[0]:
      file_name = r3['metadata']['file_name']
      if r3['score'] >=0.3:
          if file_name not in final_results:
              final_results[file_name] = {}
              final_results[file_name]['candidate_name'] = r3['metadata']['full_name']
              final_results[file_name]['metadata'] = r3['metadata']
              final_results[file_name]['score'] = [r3['score']]
              final_results[file_name]['values'] = [r3['values']]
              final_results[file_name]['metadata']['highest_degree'] = final_results[file_name]['metadata']['text']
              final_results[file_name]['reason'] = f"The skills matched with the skills of {r3['metadata']['full_name']}"
          else:
              final_results[file_name]['score'].append(r3['score'])
              final_results[file_name]['values'].append(r3['values'])
              final_results[file_name]['reason'] += "\n" + f"The skills matched with the skills of {r3['metadata']['full_name']}"

  return final_results



In [ ]:
query_results= await filtering_skill_edu_exp(job_data,index)

In [ ]:
job_data['required_experience']

'4+ years experience in supporting hosted services in a high-volume customer-facing environment, 3+ years proven experience in running and managing infrastructure at scale using container orchestration tool kubernetes'

In [ ]:
job_data['required_education']

"Bachelor's Degree in Computer Science, Engineering or equivalent field"

In [ ]:
job_data['required_skills']

'Experience in building large CI/CD processes (experience with Github actions & Jenkins is preferred), Experience with DevOps tools, processes, culture, distributed systems, supporting mission-critical systems, Experience working on Continuous Integration / Delivery pipelines using tools such as Jenkins and Github actions, Demonstrated ability to write programs / utilities using a high-level programming language like: Go, Ruby, Python as well as Bash, Experience in performing migration from traditional technology stack(s) to AWS EKS or similar'

In [ ]:
for key, value in query_results.items():
    print(f"file name: {key}")
    print(f"candidate name: {value['candidate_name']}")
    print(f"text: {value['metadata']['text']}")
    print(f"score: {value['score']}")
    print(f"reason: {value['reason']}")
    print("\n")

file name: /content/CV_folder/H.M. Saqlain Mushtaque Resume 2023.pdf
candidate name: H.M. Saqlain Mushtaque
text: ['Python', 'SQL and NoSQL', 'Linux', 'GitHub', 'Data Structure and Algorithm', 'Data Analysis', 'CI/CD', 'REST API', 'Docker', 'MS Office', 'AWS Services', 'Problem Solver']
score: [0.33959043, 0.35921666, 0.405538887]
reason: The experience matched with the experience of H.M. Saqlain Mushtaque
The education matched with the education of H.M. Saqlain Mushtaque
The skills matched with the skills of H.M. Saqlain Mushtaque


file name: /content/CV_folder/SM Ghuffran 2022.pdf
candidate name: SYED MUHAMMAD GHUFFRAN
text: ['Embedded Systems: RTOS, Microcontrollers (ARM, ESP32), IoT protocols (MQTT, CoAP), Device Drivers, Embedded Linux, UART/SPI/I2C', 'Programming Languages: C/C++, Python (Intermediate), Assembly Language, Java, MATLAB', 'Frameworks & Libraries: Node.js, React, Flask, TensorFlow (Basic), OpenCV', 'Development Tools: Git, Docker, Jenkins, JIRA, Bitbucket, Visual S

In [ ]:
len(query_results)

0

# Ingnore from there

## Checking
- 1